In [1]:
import sys
sys.path.insert(0, "./src")

import f5_tts.model
print("f5_tts.model contents:", dir(f5_tts.model))


/disk4/github/wrightmikea/DMOSpeech2/.venv/lib/python3.10/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.575 seconds.
Prefix dict has been built successfully.


Word segmentation module jieba initialized.



/disk4/github/wrightmikea/DMOSpeech2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


f5_tts.model contents: ['CFM', 'DiT', 'MMDiT', 'Trainer', 'UNetT', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'backbones', 'cfm', 'dataset', 'modules', 'trainer', 'utils']


In [2]:
import duration_predictor
print("duration_predictor contents:", dir(duration_predictor))


duration_predictor contents: ['PositionalEncoding', 'SpeechLengthPredictor', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'calculate_remaining_lengths', 'nn', 'torch']


In [3]:
import guidance_model
print("guidance_model contents:", dir(guidance_model))


guidance_model contents: ['Callable', 'ConformerCTC', 'ConformerDiscirminator', 'DiT', 'ECAPA_TDNN', 'F', 'Guidance', 'NoOpContext', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_kl_dist_func', 'annotations', 'default', 'exists', 'lens_to_mask', 'list_str_to_idx', 'list_str_to_tensor', 'mask_from_frac_lengths', 'nn', 'np', 'predict_flow', 'random', 'torch']


In [4]:
import os
print("Current directory explicitly is:", os.getcwd())
#print("Explicit contents of ckpts folder:", os.listdir('ckpts'))

Current directory explicitly is: /disk4/github/wrightmikea/DMOSpeech2/src


In [5]:
import sys
sys.path.insert(0, "./src")

from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse, JSONResponse
import torchaudio
import torch
import uuid
import nest_asyncio
import uvicorn

from transformers import AutoTokenizer
from unimodel import UniModel
from duration_predictor import SpeechLengthPredictor
from guidance_model import Guidance

nest_asyncio.apply()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Explicit tokenizer initialization
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
prompt_tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# Explicit correct model instantiation from original notebook logic
mel_model = UniModel(model='cfm').to(DEVICE)
mel_model.load_state_dict(torch.load("../ckpts/model_85000.pt"))
mel_model.eval()

duration_model = SpeechLengthPredictor().to(DEVICE)
duration_model.load_state_dict(torch.load("../ckpts/model_1500.pt"))
duration_model.eval()

guidance_model = Guidance().to(DEVICE)
guidance_model.eval()

app = FastAPI()

style_state = {"wav": None, "prompt_ids": None}

@app.post("/init_voice")
async def init_voice(audio_file: UploadFile = File(...), reference_text: str = Form(...)):
    wav, sr = torchaudio.load(audio_file.file)
    if sr != 22050:
        wav = torchaudio.functional.resample(wav, orig_freq=sr, new_freq=22050)
    style_state["wav"] = wav.unsqueeze(0).to(DEVICE)
    style_state["prompt_ids"] = prompt_tokenizer.encode(reference_text, return_tensors="pt").to(DEVICE)
    return JSONResponse({"status": "Voice style initialized."})

@app.post("/generate_audio")
async def generate_audio(target_text: str = Form(...)):
    if style_state["wav"] is None or style_state["prompt_ids"] is None:
        return JSONResponse({"error": "Initialize voice first."}, status_code=400)

    tgt_ids = tokenizer.encode(target_text, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        duration_pred = duration_model(style_state["wav"], style_state["prompt_ids"])
        generated_mel = mel_model.infer(
            style_state["wav"], style_state["prompt_ids"], tgt_ids,
            duration_pred, guidance_model
        )

    waveform = torchaudio.functional.griffinlim(generated_mel.squeeze().cpu(), n_fft=1024)
    filename = f"/tmp/{uuid.uuid4().hex}.wav"
    torchaudio.save(filename, waveform.unsqueeze(0), 22050)

    return FileResponse(filename, media_type='audio/wav', filename="generated.wav")

uvicorn.run(app, host="0.0.0.0", port=8000)


AttributeError: 'str' object has no attribute 'time_embed2'